# Notes on Topic 18 and 19

## Topic 18 Intro to Ling Reg

#### Coeff. of Deter.
- "R-Squared": =statistical **measure** that is used to assess the **goodness of fit** of a regression model
- "R^2"% of the variations in y are explained by the X in our model.

#### Regression Lab

In [ ]:
def plot_reg(X, Y, Y_pred):
    plt.scatter(X, Y, label='data')
    plt.plot(X, Y_pred, label='regression line')

#### Assumptions for LinReg
- Linearity: requires that there is a linear relationship between y and X. Linear means that the change in Y by 1-unit change in X, is constant
    - Can best be tested with scatter plots
- Normality: model residuals should follow a normal distribution
    - Check with histograms or a Q-Q-Plots
- Homoscedasticity (AKA homoskedasticity): y's variability is equal across the range of values of X
    - Check with scatter plot with y on y-axis, or use significance tests like Breusch-Pagan / Cook-Weisberg test or White general test to detect this phenomenon. Remember that, if these tests give you a p-value < 0.05, the null hypothesis can rejected, and you can assume the data is heteroscedastic.
- check for Linearity and Homoscedasticity in the residuals as well.

#### OLS in Statsmodels
- for model summary shown on bottom of this page
- sm.graphics.plot_regress_exog(model, "height", fig=fig) plots shown on bottom of page

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.formula.api import ols
model = ols(formula=f, data=df).fit()
model.summary()

import statsmodels.api as sm
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "height", fig=fig)
# This plots four graphs in a 2 by 2 figure: ‘endog versus exog’,
# ‘residuals versus exog’, ‘fitted versus exog’ and ‘fitted plus residual versus exog’

import scipy.stats as stats
residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

#### OLS in Statsmodels Lab
- Step 1: Read the dataset and inspect its columns and 5-point statistics
- Step 2: Plot histograms with kde overlay to check the distribution of the predictors
- Step 3: Test for the linearity assumption 
- Step 4: Run a simple regression in Statsmodels with TV as a predictor and get Reg Diagnostics summary
- Step 5: Draw a prediction line with data points on a scatter plot for X (TV) and Y (Sales)
- Step 6: Visualize the error term for variance and heteroscedasticity
- Step 7: Check the normality assumptions by creating a QQ-plot
- Step 8: Repeat the above for radio and record your observations

In [ ]:
# Step 2
for column in data:
    data[column].plot.hist(density=True, label = column+' histogram')
    data[column].plot.kde(label =column+' kde')
    plt.legend()

# Step 3
fig, axs = plt.subplots(1, 3, sharey=True, figsize=(18, 6))
for idx, channel in enumerate(['TV', 'radio', 'newspaper']):
    data.plot(kind='scatter', x=channel, y='sales', ax=axs[idx], label=channel)
plt.legend()

# Step 4
import statsmodels.formula.api as smf
# create a fitted model in one line
model = smf.ols(formula=f, data=data).fit()
model.summary()

# Step 5
# create a DataFrame with the minimum and maximum values of TV
X_new = pd.DataFrame({'TV': [data.TV.min(), data.TV.max()]})
print(X_new.head())
# make predictions for those x values and store them
preds = model.predict(X_new)
# first, plot the observed data and the least squares line
data.plot(kind='scatter', x='TV', y='sales')
plt.plot(X_new, preds, c='red', linewidth=2)

# Step 6
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "TV", fig=fig)

# Step 7
residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)

#### Regression Diagnostics in Statsmodels
- Normality check Q-Q plots (AKA normal density plots)
    - can relate q-q plots to hist/kde's
- Normality check (Jarque-Bera Test)
    - inspects the skewness and kurtosis of data to see if it matches a normal distribution
    - Q-Q Plots can become unreliable when your sample size is large
    - Null Hypo is Normality
- Heteroscedasticity check (Goldfeld-Quandt test AKA GQ test)
    - used in regression analysis to check for homoscedasticity in the error terms
    - checks if you can define a point that can be used to **differentiate** the variance of the error term
    - assumes data is normally distributed, so check before
    - null hypothesis is homoscedasticity, larger F-stat means more evidence against null hypothesis and more likely we see different variance for the two groups
    - Null Hypo is Homoscedasticity
- Heteroscedasticity check (Breush-Pagan Test and White’s Test)
    - Statsmodel also offers newer tests for heteroscadasticity including the [Breush-Pagan Test](https://www.statsmodels.org/devel/generated/statsmodels.stats.diagnostic.het_breuschpagan.html) and [White’s Test](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.het_white.html#statsmodels.stats.diagnostic.het_white) which may be advantageous in certain cases.

##### Checking for Linearity: Joint Plot

In [ ]:
sns.jointplot(x= <column>, y= <column>, data=<dataset>, kind='reg')

##### Checking for Normality: QQ-plot and JB Test

In [ ]:
import statsmodels.api as sm
import scipy.stats as stats
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
# How to run JB test on its own/ also in model summary
name = ['Jarque-Bera','Prob','Skew', 'Kurtosis']
test = sms.jarque_bera(model.resid)
list(zip(name, test))

##### Checking for Homoscedasticity: GQ Test and viewing scatter plot
In the image below, you can see how observations are split into two groups. Next, a test statistic is run through taking the ratio of mean square residual errors for the regressions on the two subsets. Evidence of heteroscedasticity is based on performing a hypothesis test (more on this later) as shown in the image.


In [ ]:
# See what GQ plot looks like
lwr_thresh = data.TV.quantile(q=.45)
upr_thresh = data.TV.quantile(q=.55)
middle_10percent_indices = data[(data.TV >= lwr_thresh) & (data.TV<=upr_thresh)].index
# len(middle_10percent_indices)

indices = [x-1 for x in data.index if x not in middle_10percent_indices]
plt.scatter(data.TV.iloc[indices], model.resid.iloc[indices])
plt.xlabel('TV')
plt.ylabel('Model Residuals')
plt.title("Residuals versus TV Feature")
plt.vlines(lwr_thresh, ymax=8, ymin=-8, linestyles='dashed',linewidth=2)
plt.vlines(upr_thresh, ymax=8, ymin=-8, linestyles='dashed',linewidth=2);

Here is a brief description of the steps involved:

* Order the data in ascending order 
* Split your data into _three_ parts and drop values in the middle part.
* Run separate regression analyses on two parts. After each regression, find the Residual Sum of Squares.
* Calculate the ratio of the Residual sum of squares of two parts.
* Apply the F-test. 

[Here](https://en.wikipedia.org/wiki/F-test) is a quick introduction to F-test

For now, you should just remember that high F values typically indicate that the variances are different. If the error term is homoscedastic, there should be no systematic difference between residuals and F values will be small.
However, if the standard deviation of the distribution of the error term is proportional to the x variable, one part will generate a higher sum of square values than the other.

In [ ]:
# Run Goldfeld Quandt test
name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(model.resid.iloc[indices], model.model.exog[indices])
list(zip(name, test))

In [ ]:
# View heteroscedaasticity
plt.scatter(model.predict(df[x_cols]), model.resid)
plt.plot(model.predict(df[x_cols]), [0 for i in range(len(df))])

##### Ex. of model refining

In [ ]:
# Ex. of removing outliers, model refining (then you would check normality and homoscadasticity like above)
#Finding a cutoff point
for i in range(85, 99):
    q = i / 100
    print('{} percentile: {}'.format(q, df['MPG_Highway'].quantile(q=q)))
subset = df[df['MPG_Highway'] < 38]
print('Percent removed:',(len(df) - len(subset))/len(df))
outcome = 'MPG_Highway'
x_cols = ['Passengers', 'Wheelbase', 'Weight', 'Fueltank']
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=subset).fit()
model.summary()

In [ ]:
df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

#### Interpreting Significance and P-values 

##### Hypothesis Testing in Regression 

During regression, you try to measure the model parameters (coefficients). The null and alternative hypotheses are also set up in those terms.
- **Null Hypothesis ($H_0$)**: There is no relationship between X and y

- **Alternative Hypothesis ($H_a$):** There is "some" relation between X and y

- p-value as a level of statistical significance
    - represents a **probability of observing your results (or something more extreme) given that the null hypothesis is true** and the probability that the coefficient is actually zero

In [ ]:
# Found in model summary P > |t|

## Topic 19 Multi Lin Reg

#### MLR
- $$ \hat y = \hat\beta_0 + \hat\beta_1 x_1 + \hat\beta_2 x_2 +\ldots + \hat\beta_n x_n $$
- fitted line, slope parameters, intercept

#### Dealing with Cat Vars
- Identifying Cat Vars
- Transforming Cat Vars
    - Label encoding: represent labels as numbers
    - Creating Dummy Vars / One Hot Encoding: create each category into a new column and have 1 or 0 (required)

In [ ]:
# scatter matrix (usually done on con. vars.)
pd.plotting.scatter_matrix(data, figsize=(12,15));
# indidual scattter plots with y on y-axis
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))
for xcol, ax in zip(['acceleration', 'displacement', 'horsepower', 'weight'], axes):
    data.plot(kind='scatter', x=xcol, y='mpg', ax=ax, alpha=0.4, color='b')
# histograms
import warnings
warnings.filterwarnings('ignore')
fig = plt.figure(figsize = (8,8))
ax = fig.gca()
data.hist(ax = ax);

# Label Encoding
origin = ['USA', 'EU', 'EU', 'ASIA','USA', 'EU', 'EU', 'ASIA', 'ASIA', 'USA']
origin_series = pd.Series(origin).astype('category')
origin_series.cat.codes
# or 
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
origin_encoded = lb_make.fit_transform(origin_series) # .astype('category') not required

# Creating Dummy Vars (w/o avoiding DV trap)
pd.get_dummies(origin_series)
# or
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
origin_dummies = lb.fit_transform(origin_series)
# You need to convert this back to a dataframe
origin_dum_df = pd.DataFrame(origin_dummies,columns=lb.classes_)
origin_dum_df

# Creating Dummy Vars (avoiding DV trap)
pd.get_dummies(origin_series, drop_first=True)
# The dropped category becomes the reference category. The coeffs resulting
# from remaining variables represent the change relative to the reference.

#### Dealing with Cat Vars Lab

In [ ]:
# Picking and plotting cat vars with y on y-axis
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16,10), sharey=True)
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']
for col, ax in zip(categoricals, axes.flatten()):
    (ames.groupby(col)               # group values together by column of interest
         .mean()['SalePrice']        # take the mean of the saleprice for each group
         .sort_values()              # sort the groups in ascending order
         .plot
         .bar(ax=ax))                # create a bar graph on the ax
    ax.set_title(col)
fig.tight_layout()
# Create dummy variables for your six categorical features
dummies = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)
ames_preprocessed = ames.drop(categoricals, axis=1)
ames_preprocessed = pd.concat([ames_preprocessed, dummies], axis=1)
ames_preprocessed.head()

#### Multicollinearity
- Identifying multicollinearity:
- address (drop necessary columns)

In [ ]:
# indetifying multicollinearity
pd.plotting.scatter_matrix(data_pred,figsize  = [9, 9]);
# or 
# save absolute value of correlation matrix as a data frame
# converts all values to absolute value
# stacks the row:column pairs into a multindex
# reset the index to set the multindex to seperate columns
# sort values. 0 is the column automatically generated by the stacking
df=data_pred.corr().abs().stack().reset_index().sort_values(0, ascending=False)
# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df['pairs'] = list(zip(df.level_0, df.level_1))
# set index to pairs
df.set_index(['pairs'], inplace = True)
#d rop level columns
df.drop(columns=['level_1', 'level_0'], inplace = True)
# rename correlation column as cc rather than 0
df.columns = ['cc']
# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
df.drop_duplicates(inplace=True)
df[(df.cc>.75) & (df.cc <1)]
# or 
abs(ames_preprocessed.corr()) > 0.75 # or annotated heat map
# or 
import seaborn as sns
sns.heatmap(data_pred.corr(), center=0); # coolwarm cmap and true annot

from statsmodels.stats.outliers_influence import variance_inflation_factor
X = df[x_cols]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(x_cols, vif))



#### Log Trans
- helps x's approach normality

In [ ]:
non_normal = ['displacement', 'horsepower', 'weight']
for feat in non_normal:
    data[feat] = data[feat].map(lambda x: np.log(x))
pd.plotting.scatter_matrix(data[x_cols], figsize=(10,12));

#### Feature Scaling and Normalization
- Popular transformations listed at the bottom

In [ ]:
acc = data['acceleration']
disp = data['displacement']
horse = data['horse']
weight = data['weight']

minmax_acc = (acc - min(acc)) / (max(acc) - min(acc)) # min max scaling
log_disp = np.log(disp)
standard_weight = (weight - np.mean(weight)) / np.sqrt(np.var(weight)) # standardization
meannorm_scaled_horse = (horse - np.mean(horse)) / (max(horse) - min(horse)) # mean normalization

data_scaled['acc'] = minmax_acc
data_scaled['disp'] = log_disp
data_scaled['horse'] = scaled_horse
data_scaled['weight'] = standard_weight
data_scaled.hist(figsize = [6, 6]);

#### Feature Scaling and Normalization Lab
- Look at hists for cont vars

In [ ]:
con_data = ames.loc[:, ((ames.dtypes != 'object') & (ames.nunique() > 20))]
fig, axes = plt.subplots(nrows=(con_data.shape[1] // 3), ncols=3, figsize=(16,40))
convars = [column for column in con_data.columns if column != 'Id']
for col, ax in zip(convars, axes.flatten()):
    ax.hist(ames[col].dropna(), bins='auto')
    ax.set_title(col)  
fig.tight_layout()
# Select non zero-inflated continuous features as ames_cont
ames_cont = ames[['LotFrontage', 'LotArea', 'YearBuilt', '1stFlrSF', 'GrLivArea', 'SalePrice']]
ames_cont.hist(figsize  = [8, 8], bins='auto');
# Perform log trans
import numpy as np
log_names = [f'{column}_log' for column in ames_cont.columns]
ames_log = np.log(ames_cont)
ames_log.columns = log_names
ames_log.hist(figsize=(10, 10), bins='auto')
fig.tight_layout();
# standardize
def standardize(feature):
    return (feature - feature.mean()) / feature.std()
features_final = ames_log.apply(standardize)
features_final.hist(figsize  = [8, 8], bins='auto');

# plotting before/after feature scaling
def plots(df, col, t):
    plt.figure(figsize=(12,5))
    plt.subplot(121)
    sns.kdeplot(df[col])
    plt.title('Before' + str(t).split('(')[0]) 
    
    plt.subplot(122)
    p1 = t.fit_transform(df[[col]]).flatten()
    sns.kdeplot(p1)
    plt.title('After' + str(t).split('(')[0])    
    
for col in df_m1_conts.columns:
    plots(df_m1_conts.copy(), col, MinMaxScaler())
    
for col in df_m1_conts.columns:
    plots(df_m1_conts.copy(), col, StandardScaler())
    
# scaling 
minmax = MinMaxScaler()
stndrd = StandardScaler()
minmax.fit_transform(df[[col]])
stndrd.fit_transform(df[[col]])


#### MLR in Statsmodels and SKLearn
- Interpretation: coeffs interpreted as "how does Y change for each additional unit X" where X is the (log- and min-max, standardized,...) transformed data matrix.
- sklearn coeffs may be different from statsmodels, but they should return equivalent results.

The extimates for the categorical variables are the same "up to a constant", the difference between the categorical variables is added in the intercept!

You can make sure to get the same result in both Statsmodels and Scikit-learn, by dropping out one of the `orig_`-levels. This way, you're essentially forcing the coefficient of this level to be equal to zero, and the intercepts and the other coefficients will be the same. 

In [ ]:
# df for model
data_ols = pd.concat([mpg, scaled_acc, scaled_weight, orig_dummies], axis= 1)
data_ols.head()
# baseline model
import statsmodels.api as sm
from statsmodels.formula.api import ols
y = 'mpg'
X = data_ols.drop('mpg', axis=1)
X_cols = "+".join(X.columns)
formula = y + "~" + X_cols
model = ols(formula= formula, data=data_train).fit()
# or 
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train).fit()
model.summary()
# or sklearn
from sklearn.linear_model import LinearRegression
y = data_ols['mpg']
linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg.coef_
linreg.intercept_
yhat = linreg.predict(X_test)
# to get same coeffs and intercepts as statsmodels
X_train = X.drop("orig_3",axis=1)
linreg.fit(X_train, y_train)
linreg.coef_
linreg.intercept_
X_cols = "+".join(predictors.columns)
formula = y + "~" + X_cols
model = ols(formula= formula, data=data_train).fit()
model.summary()

#### Inference vs. Prediction
- Inference: How does X affect y?
    - figure out which features affect your outcome and how your outcome changes when these features change
    - focused on only a subset of features
    - great emphasis is given to the coefficients of these features as opposed to the overall model accuracy
    - choose simpler, interpretable models like models: logistic regression, decision trees, linear SVMs 
- Prediction: How well can I use X to predict y?
    - less concerned about how and which features impact Y as opposed to how you can efficiently use them to predict Y
    - typically use all available features (and most likely engineer new features)
    - less concerned about the coefficients of these features and instead focus on the overall model accuracy
    - typically choose more complex, less interpretable models like SVMs with radial kernels, random forests, neural networks, and other techniques such regularization, cross-validation, grid search

#### Model Fit in Lin Reg
- $R^2$ increases each time we add a new predictor -- even if this new predictor doesn't add any new information to the model. Model tends to overfit if we only use $R^2$ as our model fitting criterion. This is why train-test split is essential and why regularization techniques are used to refine more advanced regression models. Make sure to read [this blogpost](https://www.statisticshowto.datasciencecentral.com/adjusted-r2/) on the difference between the two to get a better sense to why use $R^2_{adj}$ !
- **Stepwise selection**: start with an empty model (which only includes the intercept), and each time, the variable that has an associated parameter estimate with the lowest p-value is added to the model (forward step). After adding each new variable in the model, the algorithm will look at the p-values of all the other parameter estimates which were added to the model previously, and remove them if the p-value exceeds a certain value (backward step). The algorithm stops when no variables can be added or removed given the threshold values. 
- **Feature ranking with recursive feature elimination**: Scikit-learn also provides a few [functionalities for feature selection](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection). Their *Feature Ranking with Recursive Feature Elimination* selects the pre-specified $n$ most important features. This means you must specify the number of features to retail. If this number is not provided, half are used by default. See [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) for more information on how the algorithm works.
- **Forward selection using adjusted R-squared**: [This resource](https://planspace.org/20150423-forward_selection_with_statsmodels/) provides code for a forward selection procedure (much like stepwise selection, but without a backward pass), but this time looking at the adjusted R-squared to make decisions on which variable to add to the model.

In [ ]:
import statsmodels.api as sm
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included
result = stepwise_selection(predictors, data_fin['mpg'], verbose=True)
print('resulting features:')
print(result)
# or RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=3)
selector = selector.fit(predictors, data_fin['mpg'])
selector.support_
selector.ranking_
estimators = selector.estimator_
print(estimators.coef_)
print(estimators.intercept_)

# Example (LAB)
result = stepwise_selection(X, y, verbose=False);
import statsmodels.api as sm
X_fin = X[result]
X_with_intercept = sm.add_constant(X_fin)
model = sm.OLS(y,X_with_intercept).fit()
model.summary()
# or
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(X, y.values.ravel()) # convert y to 1d np array to prevent DataConversionWarning
selector.support_
selected_columns = X.columns[selector.support_]
linreg.fit(X[selected_columns],y)
yhat = linreg.predict(X[selected_columns])
SS_Residual = np.sum((y-yhat)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1 - SS_Residual/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X[selected_columns].shape[1]-1)

#### Regression Model Validation
Residuals:

$r_{i,train} = y_{i,train} - \hat y_{i,train}$ 

$r_{i,test} = y_{i,test} - \hat y_{i,test}$ 

To get a summarized measure over all the instances in the test set and training set, a popular metric is the (Root) Mean Squared Error:

RMSE = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat y_{i})^2}$

MSE = $\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat y_{i})^2$

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), len(X_test), len(y_train), len(y_test))
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test = np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)
print('RMSE Train:', np.sqrt(mse_train))
print('RMSE Test:', np.sqrt(mse_test))
# or
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse
print('RMSE Train:', np.sqrt(train_mse))
print('RMSE Test:', np.sqrt(test_mse))
r2_score(y_test, y_hat_test)
      
      
# LAB
# Evaluate the effect of train-test split size

import random
random.seed(110)

train_err = []
test_err = []
t_sizes = list(range(5,100,5))
for t_size in t_sizes:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size/100)
    linreg.fit(X_train, y_train)
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_err.append(mean_squared_error(y_train, y_hat_train))
    test_err.append(mean_squared_error(y_test, y_hat_test))
plt.scatter(t_sizes, train_err, label='Training Error')
plt.scatter(t_sizes, test_err, label='Testing Error')
plt.legend()
      
# or cell below
      

Repeat the previous example, but for each train-test split size, generate 10 iterations of models/errors and save the average train/test error. This will help account for any particularly good/bad models that might have resulted from poor/good splits in the data

In [ ]:
random.seed(900)
train_err = []
test_err = []
t_sizes = range(5,100,5)
for t_size in t_sizes:
    temp_train_err = []
    temp_test_err = []
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size/100)
        linreg.fit(X_train, y_train)
        y_hat_train = linreg.predict(X_train)
        y_hat_test = linreg.predict(X_test)
        temp_train_err.append(mean_squared_error(y_train, y_hat_train))
        temp_test_err.append(mean_squared_error(y_test, y_hat_test))
    train_err.append(np.mean(temp_train_err))
    test_err.append(np.mean(temp_test_err))
plt.scatter(t_sizes, train_err, label='Training Error')
plt.scatter(t_sizes, test_err, label='Testing Error')
plt.legend()


#### Intro to Cross Validation + Lab
When using train-test split, random samples of data are created for the training and the test set. The problem with this is that the training and test MSE strongly depend on how the training and test sets were created.

There are many ways to perform cross-validation, and we strongly recommend you have a look at the [Cross-validation documentation in Scikit-Learn](http://scikit-learn.org/stable/modules/cross_validation.html) and 
[scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

In [ ]:
from sklearn.model_selection import cross_val_score
cv_5_results  = np.mean(cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_10_results = np.mean(cross_val_score(linreg, X, y, cv=10, scoring='neg_mean_squared_error'))
cv_20_results = np.mean(cross_val_score(linreg, X, y, cv=20, scoring='neg_mean_squared_error'))
cv_5_results.mean()
# or
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
mse = make_scorer(mean_squared_error)
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
cv_5_results

#### Coeff. of Deter.
- "R-Squared": =statistical **measure** that is used to assess the **goodness of fit** of a regression model
- R^2% of the variations in y are explained by the X in our model.

## Extra Notes

#### model.summary()

Here is a brief description of these measures:

The left part of the first table gives some specifics on the data and the model:

* **Dep. Variable**: Singular. Which variable is the point of interest of the model
* **Model**: Technique used, an abbreviated version of Method (see methods for more).
* **Method**: The loss function optimized in the parameter selection process. Least Squares since it picks the parameters that reduce the training error. This is also known as Mean Square Error [MSE].
* **No. Observations**: The number of observations used by the model, or size of the training data.
* **Degrees of Freedom Residuals**: Degrees of freedom of the residuals, which is the number of observations – number of parameters. Intercept is a parameter. The purpose of Degrees of Freedom is to reflect the impact of descriptive/summarizing statistics in the model, which in regression is the coefficient. Since the observations must "live up" to these parameters, they only have so many free observations, and the rest must be reserved to "live up" to the parameters' prophecy. This internal mechanism ensures that there are enough observations to match the parameters.
* **Degrees of Freedom Model**: The number of parameters in the model (not including the constant/intercept term if present)
* **Covariance Type**: Robust regression methods are designed to be not overly affected by violations of assumptions by the underlying data-generating process. Since this model is Ordinary Least Squares, it is non-robust and therefore highly sensitive to outliers.

The right part of the first table shows the goodness of fit: 

* **R-squared**: The coefficient of determination, the Sum Squares of Regression divided by Total Sum Squares. This translates to the percent of variance explained by the model. The remaining percentage represents the variance explained by error, the E term, the part that model and predictors fail to grasp.
* **Adj. R-squared**: Version of the R-Squared that penalizes additional independent variables. 
* **F-statistic**: A measure of how significant the fit is. The mean squared error of the model divided by the mean squared error of the residuals. Feeds into the calculation of the P-Value.
* **Prob (F-statistic) or P-Value**: The probability that a sample like this would yield the above statistic, and whether the model's verdict on the null hypothesis will consistently represent the population. Does not measure effect magnitude, instead measures the integrity and consistency of this test on this group of data.
* **Log-likelihood**: The log of the likelihood function.
* **AIC**: The Akaike Information Criterion. Adjusts the log-likelihood based on the number of observations and the complexity of the model. Penalizes the model selection metrics when more independent variables are added.
* **BIC**: The Bayesian Information Criterion. Similar to the AIC, but has a higher penalty for models with more parameters. Penalizes the model selection metrics when more independent variables are added.

The second table shows the coefficient report: 

* **coef**: The estimated value of the coefficient. By how much the model multiplies the independent value by.
* **std err**: The basic standard error of the estimate of the coefficient. Average distance deviation of the points from the model, which offers a unit relevant way to gauge model accuracy.
* **t**: The t-statistic value. This is a measure of how statistically significant the coefficient is.
* **P > |t|**: P-value that the null-hypothesis that the coefficient = 0 is true. If it is less than the confidence level, often 0.05, it indicates that there is a statistically significant relationship between the term and the response.
* **[95.0% Conf. Interval]**: The lower and upper values of the 95% confidence interval. Specific range of the possible coefficient values.

The third table shows information about the residuals, autocorrelation, and multicollinearity: 

* **Skewness**: A measure of the symmetry of the data about the mean. Normally-distributed errors should be symmetrically distributed about the mean (equal amounts above and below the line). The normal distribution has 0 skew.
* **Kurtosis**: A measure of the shape of the distribution. Compares the amount of data close to the mean with those far away from the mean (in the tails), so model "peakiness". The normal distribution has a Kurtosis of 3, and the greater the number, the more the curve peaks.
* **Omnibus D’Angostino’s test**: Provides a combined statistical test for the presence of skewness and kurtosis.
* **Prob(Omnibus)**: The above statistic turned into a probability
* **Jarque-Bera**: A different test of the skewness and kurtosis
* **Prob (JB)**: The above statistic turned into a probability
* **Durbin-Watson**: A test for the presence of autocorrelation (that the errors are not independent), which is often important in time-series analysis
* **Cond. No**: A test for multicollinearity (if in a fit with multiple parameters, the parameters are related to each other).


#### fig = sm.graphics.plot_regress_exog(model, "height", fig=fig)

For the four graphs we see above:

* The **Y and Fitted vs. X** graph plots the dependent variable against our predicted values with a confidence interval. The positive relationship shows that height and weight are correlated, i.e., when one variable increases the other increases.

* The **Residuals versus height** graph shows our model's errors versus the specified predictor variable. Each dot is an observed value; the line represents the mean of those observed values. Since there's no pattern in the distance between the dots and the mean value, the OLS assumption of homoskedasticity holds.

* The **Partial regression plot** shows the relationship between height and weight, taking in to account the impact of adding other independent variables on our existing height coefficient. You'll later learn how this same graph changes when you add more variables.

* The **Component and Component Plus Residual (CCPR)** plot is an extension of the partial regression plot. It shows where the trend line would lie after adding the impact of adding our other independent variables on the weight.

#### Extra code

In [ ]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

#### Popular Transformations
Log transformation

As seen in the previous lesson, a log transformation is a very useful tool when you have data that clearly does not follow a normal distribution. Log transformation can help reduce skewness when you have skewed data, and can help reducing variability of data. 


Min-max scaling

When performing min-max scaling, you can transform x to get the transformed $x'$ by using the formula:

$$x' = \dfrac{x - \min(x)}{\max(x)-\min(x)}$$

This way of scaling brings all values between 0 and 1. 

Standardization

When 

$$x' = \dfrac{x - \bar x}{\sigma}$$

x' will have mean $\mu = 0$ and $\sigma = 1$

Note that standardization does not make data $more$ normal, it will just change the mean and the standard error!

Mean normalization
When performing mean normalization, you use the following formula:
$$x' = \dfrac{x - \text{mean}(x)}{\max(x)-\min(x)}$$

The distribution will have values between -1 and 1, and a mean of 0.

Unit vector transformation
 When performing unit vector transformations, you can create a new variable x' with a range [0,1]:
 
$$x'= \dfrac{x}{{||x||}}$$


Recall that the norm of x $||x||= \sqrt{(x_1^2+x_2^2+...+x_n^2)}$

Scikit-learn provides automatic tools to scale features, see, among others, `MinMaxScaler`, `StandardScaler`, 
and `Normalizer`. Have a look at these built-in functions and some code examples here: http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing!

#### General workflow
**O**btain Data:
- quick preview of data and business understanding

**S**crub/Clean Data:
- Drop duplicate rows/id's appropriately
- Deal w/ Data Types:
    - investigate string objects like value counts, hidden missing values, etc.
    - dates
- Deal with missing values and fill appropriately
- Deal w/ outliers and extraneous values
    - check descriptive stats
- Bin/cut cat vars and create new features if necessary
    - check distributions/hists
- Investigate/deal with multicollinearity
    - remove columns if necessary
- Save cleaned data to csv

**E**DA:
- Load clean data/preview
- Explore y variable for outliers and distribution (with boxplot and/or histogram)
    - address issues if necessary
- Check heatmap for multicollinearity and strong correlations b/w price and the features
    - note strong/weak feature candidates
- Split continuous and categorical variables
- Explore strong feature candidates with a scatter matrix
    - focus on distributions (diagonal) and scatter plots with y on y-axis

**M**odeling:
- Feature Select
- OHE cat vars
- transform cont vars
- Fit the model
- Check assumptions
    - multicollinearity
        - check variation inflation factors or heat map
            -  variance inflation factor shows if there is multicollinearity between our variables
            - vif of 5 or greater (or more definitively 10 or greater) are displaying multicollinearity with other variables
            - removing multicollinear features may hurt model performance
    - linearity
        - check scatter matrix or joint plot
    - normality
        - QQ-plot or JB test
    - homsocedasticity
        - scatterplot or GQ test (assumes normality
- repeat the process by building a model with different features or refining the current model

I**N**terpret results:
- Interpret statistics appropriately (R*2, errors, tests, coeffs, etc.)

